## 디코더의 첫번째 서브층 : 셀프 어텐션과 룩-어헤드 마스크

In [1]:
import torch
import math, copy, time
import torch.nn as nn
import torch.nn.functional as F

참고문헌  
\[1\] [16-01 트랜스포머(Transformer)](https://wikidocs.net/31379)  
\[2\] [The Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html#prelims)  
\[3]] [Understanding Masking in PyTorch for Attention Mechanisms](https://medium.com/@swarms/understanding-masking-in-pytorch-for-attention-mechanisms-e725059fd49f)  

![fig1](../images/decoder._look_a_head.png)

위 그림과 같이 디코더도 인코더와 동일하게 임베딩 층과 포지셔널 인코딩을 거친 후의 문장 행렬이 입력됨.  
트랜스포머 또한 seq2seq와 마찬가지로 교사 강요(Teacher Forching)을 사용하여 훈련되므로 학습 과정에서  
디코더는 번역할 문장에 해당되는 **\<sos\> je suis étudiant** 의 문장 행렬을 한 번에 입력받습니다.  
그리고 디코더는 이 문장 행렬로부터 각 시점의 단어를 예측하도록 훈됩니다.   

여기서 문제가 있습니다. seq2seq의 디코더에 사용되는 RNN 계열의 신경망은 입력 단어를 매 시점마다   
순차적으로 입력받으므로 다음 단어 예측에 현재 시점을 포함한 이전 시점에 입력된 단어들만 참고할 수 있습니다.  
반면, 트랜스포머는 문장 행렬로 입력을 한 번에 받으므로 현재 시점의 단어를 예측하고자 할 때,  
입력 문장 행렬로부터 미래 시점의 단어까지도 참고할 수 있는 현상이 발생합니다. 가령, **suis**를 예측해야 하는  
시점이라고 해봅시다. RNN 계열의 seq2seq의 디코더라면 현재까지 디코더에 입력된 단어는 **\<sos\>**와 **je**뿐일 것입니다.  
반면, 트랜스포머는 이미 문장 행렬로 **\<sos\> je suis étudiant**를 입력받았습니다.

이를 위해 트랜스포머의 디코더에서는 현재 시점의 예측에서 현재 시점보다 미래에 있는 단어들을 참고하지  
못하도록 **룩-어헤드 마스크 \(look-ahead mask\)** 를 도입했습니다. 직역하면 **미리보기에 대한 마스크**입니다.


![fig_2](../images/decoder_mask1.png)

**룩-어헤드 마스크**(look-ahead mask)는 디코더의 첫번째 서브층에서 이루어집니다.  
디코더의 첫번째 서브층인 멀티 헤드 셀프 어텐션 층은 인코더의 첫번째 서브층인  
멀티 헤드 셀프 어텐션 층과 동일한 연산을 수행합니다. 오직 다른 점은 어텐션  
스코어 행렬에서 마스킹을 적용한다는  점만 다릅니다.  
우선 다음과 같이 셀프 어텐션을 통해 어텐션 스코어 행렬을 얻습니다.  
![](../images/decoder_attention_score_matrix.png)  

이제 자기 자신보다 미래에 있는 단어들을 참고하지 못하도록 다음과 같이 마스킹합니다.  

![](../images/look_ahead_mask.png)

마스킹 된 후의 어텐션 스코어 행렬의 각 행을 보면 자기 자신과 그 이전 단어들만을 참고할  
수 있음을 볼 수 있음. 그 외에는 근본적으로 셀프 어텐션이라는 점과 멀티 헤드 어텐션을  
수행한다는 점에서 인코더의 첫번째 서브층과 같음.  



### Look-ahead Mask
Look-ahead masks prevent the model from looking at future tokens.

**torch.triu**는 PyTorch에서 행렬의 상삼각 행렬(upper triangular matrix) 을 추출하거나 생성하는 데  
사용되는 함수입니다. 이 함수는 행렬의 대각선과 그 위쪽 요소를 유지하고, 나머지 요소를 0으로 설정합니다.

In [2]:
def create_look_ahead_mask(size):
    mask = torch.triu(torch.ones(size, size), diagonal=1)
    return mask  # (seq_len, seq_len)

In [3]:
# Example usage
look_ahead_mask = create_look_ahead_mask(4)
print(look_ahead_mask)

tensor([[0., 1., 1., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.]])


In [6]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores += (mask*-1e9)
        print(scores)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn


``` python
scores += (mask*-1e9)
```

- scores: shape (batch_size, num_heads, seq_len, seq_len)의 어텐션 score 행렬입니다.
- mask: 일반적으로 0과 1 (또는 0과 -inf로 변환 가능한 값)을 가지는 tensor로, 어떤 위치의 어텐션을 막을지 지정합니다.
  - mask가 1이면 가려야 할 위치
  - mask가 0이면 허용되는 위치  
-1e9: 매우 큰 음수. softmax 전에 더해지면 softmax 결과가 0에 수렴하게 만듭니다.

예)

``` python
scores = [[2.0, 1.0, 3.0]]
mask   = [[0,   1,   0]]
```

mask * -1e9 = [[0, -1e9, 0]]  
→ scores += mask * -1e9  
→ [[2.0, -1e9, 3.0]]  
→ softmax 결과는 [0.119, ~0, 0.880]처럼 2번째 값이 거의 0이 됩니다.  
즉, **softmax를 통해 해당 위치로의 attention이 사실상 불가능하도록** 만듭니다.  

**왜 +=를 쓰는가?**

기존의 score값에 패널티를 부여하는 방식이기 때문임.  
- 더하기(+=)를 통해 기존 score에서 특정 위치를 제외하고 나머지는 그대로 유지됨.
- 이 방식은 **broadcasting**으로 쉽게 적용되며 계산이 효율적임.


**왜 마스크된 위치만 영향을 받는가?**
- mask와 scores는 동일 shape이므로 같은 위치끼리 더해진다.
- mask가 0인 위치는 0 * -1e9 = 0 → 영향 없음.
- mask가 1인 위치는 1 * -1e9 = -1e9 → 해당 위치의 score에 매우 큰 패널티가 생김.
- 따라서, +=는 **"mask가 설정된 위치에만 -1e9를 더하는 것"** 이 됩니다.  
  
**+=는 element-wise 연산임**  

그리고 $-1e9 (-10^9)$는 매무큰 음수임

---

| 위치 | scores 값 | mask × -1e9 | 결과                       |
|:------|:-----------|:-------------|:----------------------------|
| 0    | 2.0       | 0           | 2.0                        |
| 1    | 1.0       | -1e9        | 1.0 - 1e9 → 매우 작은 값   |
| 2    | 3.0       | 0           | 3.0                        |


In [7]:
# Example usage
d_model = 512
batch_size = 2
seq_len = 4

q = torch.rand((batch_size, seq_len, d_model))
k = torch.rand((batch_size, seq_len, d_model))
v = torch.rand((batch_size, seq_len, d_model))
mask = create_look_ahead_mask(seq_len)

attention_output, attention_weights = attention(q, k, v, mask)
print(attention_weights)

tensor([[[ 5.4595e+00, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [ 5.8698e+00,  5.8352e+00, -1.0000e+09, -1.0000e+09],
         [ 5.4011e+00,  5.4265e+00,  5.6409e+00, -1.0000e+09],
         [ 5.4078e+00,  5.6980e+00,  5.9136e+00,  5.5877e+00]],

        [[ 5.8050e+00, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [ 5.5221e+00,  5.8155e+00, -1.0000e+09, -1.0000e+09],
         [ 5.6259e+00,  5.9072e+00,  5.7836e+00, -1.0000e+09],
         [ 5.8311e+00,  6.0171e+00,  5.9810e+00,  5.7728e+00]]])
tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.5087, 0.4913, 0.0000, 0.0000],
         [0.3033, 0.3111, 0.3855, 0.0000],
         [0.1926, 0.2574, 0.3194, 0.2306]],

        [[1.0000, 0.0000, 0.0000, 0.0000],
         [0.4272, 0.5728, 0.0000, 0.0000],
         [0.2861, 0.3790, 0.3349, 0.0000],
         [0.2320, 0.2795, 0.2696, 0.2189]]])
